<a href="https://colab.research.google.com/github/ThalitaCoelhoAguiar/meus-notebooks-colab/blob/main/Trab2_Top2_Thalita.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Trabalho 2 Tópicos
### Thalita Coelho Aguiar/ 222030324


In [ ]:
#Importação de Bibliotecas

!pip install -q tensorflow-datasets tensorflow

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import re
import string
from tensorflow.keras.layers import Embedding, Dense, LayerNormalization, Dropout, MultiHeadAttention
from tensorflow.keras.preprocessing.sequence import pad_sequences

#Importação dos dados
Os dados se referem a um par de frases em português e francês.

In [ ]:
# Carregar o dataset
dataset_name = "ted_hrlr_translate/fr_to_pt"
builder = tfds.builder(dataset_name) # Construir Dataset
builder.download_and_prepare()
dataset = builder.as_dataset(as_supervised=True) # Convertendo para um Dataset do TensorFlow
train_data, val_data = dataset['train'], dataset['validation'] # Obtém os dados de treino e de teste

# Pré-processar o texto usando TensorFlow
def preprocess_text(text):
    text = tf.strings.lower(text) #Deixa todas palavras minusculas
    text = tf.strings.regex_replace(text, f"[{string.punctuation}]", "") #retira pontuação
    text = tf.strings.strip(text) # Remove espaços em branco no inicio e no fim
    return text

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/43873 [00:00<?, ? examples/s]

...

Generating validation examples...:   0%|          | 0/1131 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ted_hrlr_translate/fr_to_pt/incomplete.DL0WOB_1.0.0/ted_hrlr_translate-val…

Generating test examples...:   0%|          | 0/1494 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ted_hrlr_translate/fr_to_pt/incomplete.DL0WOB_1.0.0/ted_hrlr_translate-tes…

Dataset ted_hrlr_translate downloaded and prepared to /root/tensorflow_datasets/ted_hrlr_translate/fr_to_pt/1.0.0. Subsequent calls will reuse this data.


# Tokenização: é o processo de dividir um texto em unidades menores chamadas "tokens". Transforma frases em números para os modelos entenderem.

In [ ]:
# Tokenização
tokenizer_fr = tf.keras.preprocessing.text.Tokenizer(filters='') #Cria tokens para frases do Francês
tokenizer_pt = tf.keras.preprocessing.text.Tokenizer(filters='')#Cria tokens para frases do Português

# Criar vocabulário
def build_tokenizer(data, tokenizer):
    sentences = [str(text.numpy()) for text, _ in data.take(10000)]  #  Converte o tensor text (do dataset) em um array NumPy.
    tokenizer.fit_on_texts(sentences) #Cria um dicionário onde cada palavra recebe um índice único.
    return tokenizer

tokenizer_fr = build_tokenizer(train_data, tokenizer_fr) # Treina o tokenizador com frases em francês do conjunto de treino
tokenizer_pt = build_tokenizer(train_data.map(lambda fr, pt: (pt, fr)), tokenizer_pt)

vocab_size_fr = len(tokenizer_fr.word_index) + 1 # Calcula o tamanho do vocabulário de francês +1
vocab_size_pt = len(tokenizer_pt.word_index) + 1# Calcula o tamanho do vocabulário de Português +1

# Converter frases para sequência de índices
def tokenize_pairs(fr, pt):
    fr = tokenizer_fr.texts_to_sequences(fr)
    pt = tokenizer_pt.texts_to_sequences(pt)
    return fr, pt

# Criar dataset tokenizado
def encode_data(data):
    fr_texts, pt_texts = [], []
    for fr, pt in data.take(10000):  # Limite para evitar problemas de memória
        fr_texts.append(str(fr.numpy()))  # Converte o tensor `fr` para string e adiciona à lista de textos em francês
        pt_texts.append(str(pt.numpy()))# Converte o tensor `pt` para string e adiciona à lista de textos em Português

    fr_seq, pt_seq = tokenize_pairs(fr_texts, pt_texts)  # Converte os textos tokenizados em sequências numéricas
    fr_seq = tf.keras.preprocessing.sequence.pad_sequences(fr_seq, padding='post') # Adiciona padding
    pt_seq = tf.keras.preprocessing.sequence.pad_sequences(pt_seq, padding='post')# Adiciona padding
    return fr_seq, pt_seq

train_fr, train_pt = encode_data(train_data) # Processa os dados de treino
val_fr, val_pt = encode_data(val_data) # Processa os dados de validação

#LSTM

In [ ]:
# Construção do modelo Encoder-Decoder com LSTM
embedding_dim = 64 # Dimensão dos vetores de embedding usados para representar palavras
units = 128 #define o número de neurônios na camada recorrente.

class Encoder(tf.keras.Model): # Define a classe Encoder baseada em uma rede neural do Keras
    def __init__(self, vocab_size, embedding_dim, units):
        super(Encoder, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim) # Camada de embedding para representar palavras como vetores contínuos
        self.lstm = tf.keras.layers.LSTM(units, return_sequences=True, return_state=True) # Camada LSTM que processa a sequência de entrada.

    def call(self, x): # Método de chamada para processar os dados de entrada
        x = self.embedding(x) # Converte os índices das palavras para embeddings
        output, state_h, state_c = self.lstm(x) # Passa pela LSTM
        return output, state_h, state_c # Retorna a saída completa da LSTM e os estados ocultos

class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, units):
        super(Decoder, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(units, return_sequences=True, return_state=True)  # Camada LSTM que processa a sequência gerada pelo Decoder
        self.fc = tf.keras.layers.Dense(vocab_size, activation='softmax') # Camada totalmente conectada para prever a próxima palavra na sequência

# Método de chamada que recebe a entrada x e os estados ocultos e de célula da LSTM

    def call(self, x, hidden_state, cell_state):
        x = self.embedding(x)
        output, state_h, state_c = self.lstm(x, initial_state=[hidden_state, cell_state])
        x = self.fc(output)
        return x, state_h, state_c

encoder = Encoder(vocab_size_fr, embedding_dim, units) # Inicializa o Encoder com o vocabulário Francês

decoder = Decoder(vocab_size_pt, embedding_dim, units) # Inicializa o Encoder com o vocabulário Português

 um embedding é uma representação vetorial contínua de palavras ou tokens. Ele transforma palavras em vetores densos de tamanho fixo, permitindo que o modelo capture relações semânticas entre elas.

estado oculto (state_h): contém a memória de longo prazo, que é mantida pela célula da LSTM para armazenar informações importantes ao longo da sequência.

e estado da célula( estado da célula): contém a informação sobre o estado atual após o processamento de cada palavra ou token

In [ ]:
# Construção do modelo de treinamento
encoder_inputs = tf.keras.Input(shape=(None,)) # Define a entrada do Encoder
encoder_outputs, state_h, state_c = encoder(encoder_inputs) #Passa a entrada pelo Encoder

decoder_inputs = tf.keras.Input(shape=(None,)) # Define a entrada do Decoder
decoder_outputs, _, _ = decoder(decoder_inputs, state_h, state_c) #Passa a entrada pelo Decoder

model = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs) # Cria o modelo de treinamento

# Compilação
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#treinamento do modelo
batch_size = 64
epochs = 3

train_dataset = tf.data.Dataset.from_tensor_slices(((train_fr, train_pt[:, :-1]), train_pt[:, 1:]))
train_dataset = train_dataset.batch(batch_size).shuffle(1000)

val_dataset = tf.data.Dataset.from_tensor_slices(((val_fr, val_pt[:, :-1]), val_pt[:, 1:]))
val_dataset = val_dataset.batch(batch_size)

model.fit(train_dataset, epochs=epochs, validation_data=val_dataset)

Epoch 1/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 1351s 9s/step - accuracy: 0.8557 - loss: 4.3845 - val_accuracy: 0.8657 - val_loss: 0.9280
Epoch 2/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 1346s 9s/step - accuracy: 0.8890 - loss: 0.7860 - val_accuracy: 0.8745 - val_loss: 0.8644
Epoch 3/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 1346s 9s/step - accuracy: 0.8943 - loss: 0.7526 - val_accuracy: 0.8751 - val_loss: 0.8494


#LSTM com Atenção

In [ ]:
# LSTM com Atenção
embedding_dim = 64
units = 128

class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, units):
        super(Encoder, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(units, return_sequences=True, return_state=True)

    def call(self, x):
        x = self.embedding(x)
        output, state_h, state_c = self.lstm(x)
        return output, state_h, state_c

class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, units):
        super(Decoder, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(units, return_sequences=True, return_state=True)
        self.fc = tf.keras.layers.Dense(vocab_size, activation='softmax')
        # Camada de Atenção
        self.attention = tf.keras.layers.Attention()

    def call(self, x, hidden_state, cell_state, encoder_output):
        x = self.embedding(x)
        output, state_h, state_c = self.lstm(x, initial_state=[hidden_state, cell_state])

        # Mecanismo de Atenção
        context_vector = self.attention([output, encoder_output])
        # Concatenação do vetor de contexto com a saída do LSTM
        output = tf.concat([output, context_vector], axis=-1)
        x = self.fc(output)
        return x, state_h, state_c

encoder = Encoder(vocab_size_fr, embedding_dim, units)
decoder = Decoder(vocab_size_pt, embedding_dim, units)

In [ ]:
# Construção do modelo de treinamento

encoder_inputs = tf.keras.Input(shape=(None,))
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

decoder_inputs = tf.keras.Input(shape=(None,))
decoder_outputs, _, _ = decoder(decoder_inputs, state_h, state_c, encoder_outputs) # Passando encoder_outputs para o decoder

model = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compilação
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Treinamento do modelo

batch_size = 64
epochs = 3 #

train_dataset = tf.data.Dataset.from_tensor_slices(((train_fr, train_pt[:, :-1]), train_pt[:, 1:]))
train_dataset = train_dataset.batch(batch_size).shuffle(1000)

val_dataset = tf.data.Dataset.from_tensor_slices(((val_fr, val_pt[:, :-1]), val_pt[:, 1:]))
val_dataset = val_dataset.batch(batch_size)

model.fit(train_dataset, epochs=epochs, validation_data=val_dataset)

Epoch 1/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 40s 220ms/step - accuracy: 0.8550 - loss: 3.3875 - val_accuracy: 0.8736 - val_loss: 0.8737
Epoch 2/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 35s 221ms/step - accuracy: 0.8945 - loss: 0.7668 - val_accuracy: 0.8776 - val_loss: 0.8588
Epoch 3/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 42s 225ms/step - accuracy: 0.8953 - loss: 0.7577 - val_accuracy: 0.8790 - val_loss: 0.8412


#Encoder-decoder utilizando Transformadores

* Encoder → Processa a frase de entrada com Self-Attention.para entender relações entre palavras.  Processa todas as palavras ao mesmo tempo (paralelamente).
* Decoder → Usa Masked Self-Attention para prever cada palavra sem olhar as futuras. Usa Cross-Attention para focar nas partes relevantes do Encoder

O modelo Encoder-Decoder baseado em Transformers substitui redes LSTM/RNNs tradicionais por camadas de atenção multi-head (Self-Attention e Cross-Attention), permitindo capturar dependências de longo alcace entre palavras sem precisar processar sequencialmente.

* Gera a tradução palavra por palavra













No caso do LSTM, o contexto era acumulado através dos estados ocultos ao longo da sequência, mas com Self-Attention, a relação entre as palavras é processada de forma paralela.



In [ ]:
# 🔹 Definição da camada TransformerEncoderLayer
class TransformerEncoderLayer(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout_rate=0.1):
        super().__init__()
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim) # Multi-Head Attention
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),   # Camada totalmente conectada com ReLU
            Dense(embed_dim)
        ])
        self.norm1 = LayerNormalization(epsilon=1e-6) # Normalização da saída
        self.norm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(dropout_rate)
        self.dropout2 = Dropout(dropout_rate)

    def call(self, x, training):
        attn_output = self.attention(x, x)  # Self-Attention no Encoder
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.norm1(x + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.norm2(out1 + ffn_output)


In [ ]:
# Definição da camada TransformerDecoderLayer
class TransformerDecoderLayer(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout_rate=0.1):
        super().__init__()
        self.self_attention = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.cross_attention = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(embed_dim)
        ])
        self.norm1 = LayerNormalization(epsilon=1e-6)
        self.norm2 = LayerNormalization(epsilon=1e-6)
        self.norm3 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(dropout_rate)
        self.dropout2 = Dropout(dropout_rate)
        self.dropout3 = Dropout(dropout_rate)

    def call(self, x, enc_output, training):
        seq_len = tf.shape(x)[1]
        mask = tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)  # Criando a máscara

        self_attn_output = self.self_attention(x, x, attention_mask=mask)  # Aplicando máscara
        self_attn_output = self.dropout1(self_attn_output, training=training)
        out1 = self.norm1(x + self_attn_output)

        cross_attn_output = self.cross_attention(out1, enc_output) # (Cross-Attention) entre a saída do Decoder e a saída do Encoder
        cross_attn_output = self.dropout2(cross_attn_output, training=training)  #Aplica Dropout para evitar overfitting
        out2 = self.norm2(out1 + cross_attn_output) # Normalização

        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        return self.norm3(out2 + ffn_output) #Retorna a saída final normalizada

In [ ]:
# Configuração do modelo
embed_dim = 128  # Dimensão do embedding
num_heads = 1  # Apenas uma cabeça de atenção
ff_dim = 256  # Dimensão da Feed-Forward
num_blocks = 1  # Apenas um bloco de Self-Attention

In [ ]:
# Criando o Encoder
encoder_inputs = tf.keras.Input(shape=(None,))
embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size_fr, output_dim=embed_dim)(encoder_inputs)

x = embedding_layer
for _ in range(num_blocks):
    x = TransformerEncoderLayer(embed_dim, num_heads, ff_dim)(x, training=True) # Add training=True or training=False
encoder_output = x

# Criando o Decoder
decoder_inputs = tf.keras.Input(shape=(None,))
embedding_layer_dec = tf.keras.layers.Embedding(input_dim=vocab_size_pt, output_dim=embed_dim)(decoder_inputs)

x = embedding_layer_dec
for _ in range(num_blocks):
    x = TransformerDecoderLayer(embed_dim, num_heads, ff_dim)(x, encoder_output, training=True) # Add training=True or training=False
decoder_output = x

# Camada Final de Previsão
output_layer = tf.keras.layers.Dense(vocab_size_pt, activation="softmax")(decoder_output)

In [ ]:
# Criando o modelo
transformer_model = tf.keras.Model([encoder_inputs, decoder_inputs], output_layer)

# Compilar o modelo
transformer_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:

batch_size = 64

# Criando o train_dataset com tuplas de inputs e targets
train_dataset = tf.data.Dataset.from_tensor_slices(((train_fr, train_pt[:, :-1]), train_pt[:, 1:])) \
    .shuffle(1000) \
    .batch(batch_size) \
    .prefetch(tf.data.AUTOTUNE)

# Criando o val_dataset com tuplas de inputs e targets
val_dataset = tf.data.Dataset.from_tensor_slices(((val_fr, val_pt[:, :-1]), val_pt[:, 1:])) \
    .batch(batch_size) \
    .prefetch(tf.data.AUTOTUNE)

# Treinando o modelo Transformer
transformer_model.fit(train_dataset, epochs=3, validation_data=val_dataset)

Epoch 1/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 1524s 10s/step - accuracy: 0.8883 - loss: 3.6276 - val_accuracy: 0.8837 - val_loss: 0.7849
Epoch 2/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 1498s 9s/step - accuracy: 0.9029 - loss: 0.6749 - val_accuracy: 0.8865 - val_loss: 0.7365
Epoch 3/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 1450s 9s/step - accuracy: 0.9080 - loss: 0.6068 - val_accuracy: 0.8867 - val_loss: 0.7266


* Todos os modelos foram modelados com lote de 64, e 3 épocas.
* Foi utilizada a mesma tokenização do dataset nos três modelos.
* O modelo LSTM teve acurácia de validação de 88.72%, LSTM (com atenção) teve acurácia de validação de 89,53% e o Transformador teve acurácia: de validação de 90,84%